In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 111.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 143.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install trl


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import json 
import torch
from datasets import Dataset, load_dataset
from trl import (setup_chat_format, 
                 DataCollatorForCompletionOnlyLM, 
                 SFTTrainer)
from peft import AutoPeftModelForCausalLM, LoraConfig, PeftConfig 
from transformers import (AutoTokenizer, 
                          AutoModelForCausalLM, 
                          TrainingArguments, 
                          BitsAndBytesConfig, 
                          pipeline, 
                          StoppingCriteria)

In [16]:
from huggingface_hub import login

login(
  token="",
  add_to_git_credential=True
)

Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [2]:
model_id = "nlpai-lab/KULLM3"

# 모델과 토크나이저 불러오기 
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation='eager'
    # load_in_8bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/784 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [3]:
dataset = load_dataset("json", data_files="../data/train_dataset_pipa.jsonl")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['texts'],
        num_rows: 140
    })
})

In [9]:
dataset["train"][0]

{'texts': {'instruction': '개인정보 보호법 제1조 "목적"에 대해서 설명하라.',
  'output': '이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자유와 권리를 보호하고, 나아가 개인 의 존엄과 가치를 구현함을 목적으로 한다.'}}

In [4]:
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules=[
            "q_proj",
            "up_proj",
            "o_proj",
            "k_proj",
            "down_proj",
            "gate_proj",
            "v_proj"],
        task_type="CAUSAL_LM",
)

In [18]:
args = TrainingArguments(
    output_dir="../output", 
    num_train_epochs=1,          
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,  
    optim="adamw_torch_fused",    
    logging_steps=100,            
    save_strategy="epoch",        
    learning_rate=2e-4,           
    bf16=True,                    
    tf32=True,                    
    max_grad_norm=0.3,            
    warmup_ratio=0.03,            
    lr_scheduler_type="constant", 
    push_to_hub=True,             
    report_to="wandb",            
)

In [19]:
# 권장: KULLM3 chat template 사용
SYSTEM_PROMPT = (
    "당신은 고려대학교 NLP&AI 연구실에서 만든 AI 챗봇입니다.\n"
    "당신의 이름은 'KULLM'으로, 한국어로는 '구름'을 뜻합니다.\n"
    "당신은 비도덕적이거나, 성적이거나, 불법적이거나 또는 사회 통념적으로 허용되지 않는 발언은 하지 않습니다.\n"
    "사용자와 즐겁게 대화하며, 사용자의 응답에 가능한 정확하고 친절하게 응답함으로써 최대한 도와주려고 노력합니다.\n"
    "질문이 이상하다면, 어떤 부분이 이상한지 설명합니다. 거짓 정보를 발언하지 않도록 주의합니다."
)

def formatting_func(batch_or_example):
    # batched=True인 경우: {"texts": [ {instruction, output}, ... ]}
    # 단일인 경우: {"texts": {instruction, output}}
    def render_one(t):
        instr = (t.get("instruction") or "").strip()
        out   = (t.get("output") or "").strip()
        messages = [
            {"role":"system", "content": SYSTEM_PROMPT},
            {"role":"user", "content": instr},
            {"role":"assistant", "content": out},
        ]
        s = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False) or ""
        return s if s.strip() else "<empty>"

    texts = batch_or_example.get("texts", None)

    # batched
    if isinstance(texts, list):
        return [render_one(t or {}) for t in texts]

    # 단일
    if isinstance(texts, dict):
        return [render_one(texts)]

    # 예외: 구조가 다를 때
    # (혹시 instruction/output이 최상위에 직접 있는 형태까지 커버)
    if "instruction" in batch_or_example or "output" in batch_or_example:
        return [render_one(batch_or_example)]

    # 마지막 방어
    return ["<empty>"]


In [20]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer,
    peft_config=peft_config,
    max_seq_length=512,
    packing=False,                 # 먼저 동작 확인
    formatting_func=formatting_func
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [21]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: javiow to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=17, training_loss=0.4293321721694049, metrics={'train_runtime': 178.269, 'train_samples_per_second': 0.785, 'train_steps_per_second': 0.095, 'total_flos': 4839318758227968.0, 'train_loss': 0.4293321721694049, 'epoch': 0.9714285714285714})